In [9]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import re

In [11]:
df_takes = pd.read_csv('../../Takes_Feb25.csv')
df_watch = pd.read_csv('../../Watch-Data_Feb25.csv')

In [13]:
df_intake = pd.read_csv('../../Intake-Surveys_Feb25.csv')
df_post = pd.read_csv('../../Post-Surveys_Feb25.csv')

In [23]:
df_takes.groupby('Cohort')
df_takes.groupby('Assignment')
df_takes = df_takes.dropna(subset=['Take Length'])

In [27]:
df_takes["BigOutliers"] = df_takes["Take Length"] > (85 + (1.5*(85-17)))

df_takes["BigOutliers"]

outlier_count = df_takes["BigOutliers"].sum()

outlier_count

/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/2185377046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_takes["BigOutliers"] = df_takes["Take Length"] > (85 + (1.5*(85-17)))


425

In [29]:
#Brainstorm for Formula
#over mean - building leaderboard based on number of minutes for takes
#Could weigh the take time with how many instances of participant in watch
#could also use activity?
#Group means and over means based on assignment and cohort to avoid any disparities in data
df_takes["overmean"] = df_takes["Take Length"] > (63)
overMean = df_takes["overmean"].sum()
print(overMean) #3691 - now need to split by cohort

df_takes["Cohort"] = df_takes["Cohort"].astype(str)  # Ensure it's categorical
df_takes["Take Length"] = pd.to_numeric(df_takes["Take Length"], errors="coerce")  # Convert values to numbers
mean_by_cohort = df_takes.groupby("Cohort")["Take Length"].mean()
print(mean_by_cohort)

3691
Cohort
ALDE 2024                               77.673846
ALDE 2024 #2                            67.788122
BetterOn Going Custom                  100.100000
BetterOn Your Own                       33.145679
BetterOn for Interviews                 14.666667
CTO's '24-'25                           83.305263
Corporate Academy September '24         64.352518
DELD 2023                               71.609467
ELA 2023                                62.688312
Executive Presence Studio               70.947500
Executive Presence Studio - Feb '25     82.230769
February 2024                           60.115789
February/March 2025                     78.719512
Greenhouse Presence Bootcamp            68.663498
June 2024                               57.722928
June 2024 Pilot                         70.458564
New Recorder Cohort                     72.066667
RTP Authentic Presence Training        104.500000
Red Hat Cloud Partners Team             64.947743
Session 1                             

/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/2346923463.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_takes["overmean"] = df_takes["Take Length"] > (63)
/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/2346923463.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_takes["Cohort"] = df_takes["Cohort"].astype(str)  # Ensure it's categorical
/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/2346923463.py:11: SettingWithCopyWarning: 
A value is try

In [32]:
#View how many people outperform the mean take time for each cohort
df_takes["abovemean"] = df_takes.apply(lambda row: row["Take Length"] > mean_by_cohort[row["Cohort"]], axis=1)
df_takes["abovemean"] = df_takes["abovemean"].astype(int)
result_table = df_takes.groupby("Cohort", as_index=False)["abovemean"].sum()
result_table.columns = ["Category", "Instances_Above_Mean"]
result_table

/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/3290375313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_takes["abovemean"] = df_takes.apply(lambda row: row["Take Length"] > mean_by_cohort[row["Cohort"]], axis=1)
/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/3290375313.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_takes["abovemean"] = df_takes["abovemean"].astype(int)


,Category,Instances_Above_Mean
0,ALDE 2024,118
1,ALDE 2024 #2,267
2,BetterOn Going Custom,4
3,BetterOn Your Own,146
4,BetterOn for Interviews,2
5,CTO's '24-'25,33
6,Corporate Academy September '24,354
7,DELD 2023,134
8,ELA 2023,332
9,Executive Presence Studio,335


In [34]:
#Why watch so many NAs for activity? one thing could be finding the value of activity, highest cumulative
#score of take length, activity, and # of occurrences in watch? We could weigh those variables based on
#how critical each value is at representing engagement

names = df_watch["Participant"].unique()
len(names) #only 363 names for almost 7000 observations
participant_counts = df_watch["Participant"].value_counts()
print(participant_counts)

Participant
diego.rodrigues@automattic.com    74
chi-hsuan.huang@automattic.com    72
jirka.malina@automattic.com       66
seiji.hara@automattic.com         63
ariel.maidana@automattic.com      53
                                  ..
lrh030121@gmail.com                1
ab1043@duke.edu                    1
christian.pile365@gmail.com        1
natekzucker@gmail.com              1
rdesai@us.ibm.com                  1
Name: count, Length: 362, dtype: int64


In [36]:
# Define a mapping from text responses to numerical values
response_mapping = {
    "1 - Not so much.": 1,
    "2- Somewhat.": 2,
    "3 - Pretty familiar": 3,
    "4 - Absolutely!": 4
}

# Apply the mapping to the relevant columns
for column in df_post.columns:
    df_post[column] = df_post[column].map(response_mapping).fillna(df_post[column])

# Initialize a dictionary to store mean scores
mean_scores = {}

# Calculate the mean score for each question using a for loop
for column in df_post.columns:
    if df_post[column].dtype in ['int64', 'float64']:
        mean_scores[column] = df_post[column].mean()

# Display the mean scores in a simple table
mean_scores_table = pd.DataFrame(list(mean_scores.items()), columns=['Question', 'Mean Score'])
print(mean_scores_table)

                                            Question  Mean Score
0   It’s easy for me to get into the headspace I ...    3.201646
1            Recording video empowers me personally.    3.522634
2        Recording video empowers me professionally.    3.559671
3       I am familiar with the power of my presence.    3.555556
4  I can easily shift from my “on camera me” to m...    3.111732


/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/1205565710.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_post[column] = df_post[column].map(response_mapping).fillna(df_post[column])


In [38]:
# Define a mapping from text responses to numerical values
response_mapping = {
    "1 - Not so much.": 1,
    "2- Somewhat.": 2,
    "3 - Pretty familiar": 3,
    "4 - Absolutely!": 4
}

# Apply the mapping to the relevant columns
for column in df_intake.columns:
    df_intake[column] = df_intake[column].map(response_mapping).fillna(df_intake[column])

# Initialize a dictionary to store mean scores
mean_scores = {}

# Calculate the mean score for each question using a for loop
for column in df_intake.columns:
    if df_intake[column].dtype in ['int64', 'float64']:
        mean_scores[column] = df_intake[column].mean()

# Display the mean scores in a simple table
mean_scores_table = pd.DataFrame(list(mean_scores.items()), columns=['Question', 'Mean Score'])
print(mean_scores_table)

                                Question  Mean Score
0  Connect to my audience when recording    1.847118
1           Elicit a variety of emotions    1.989924
2                 Empowers me personally    2.103015
3             Empowers me professionally    2.190955
4        Familiar with power of presence    2.137845


/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/2956094236.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_intake[column] = df_intake[column].map(response_mapping).fillna(df_intake[column])
/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/2956094236.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_intake[column] = df_intake[column].map(response_mapping).fillna(df_intake[column])
/var/folders/1w/hq9s5gxs7gs8czfbrbsb7nqc0000gn/T/ipykernel_38964/2956094236.py:11: FutureWarning: Downcasting object dtype arrays 